In [1]:
import pandas as pd


covid_data = pd.read_csv('covid_data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [2]:
vaccinations_data = pd.read_csv('covid_data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


In [3]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

In [4]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [5]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

In [6]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [7]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

In [8]:
display(covid_data['date'].min())
display(covid_data['date'].max())

display(vaccinations_data['date'].min())
display(vaccinations_data['date'].max())

Timestamp('2020-01-22 00:00:00')

Timestamp('2021-05-29 00:00:00')

Timestamp('2020-12-02 00:00:00')

Timestamp('2021-09-06 00:00:00')

In [9]:
merge_mask = ('2020-01-22' <= vaccinations_data['date']) & (vaccinations_data['date'] <= '2021-05-29')
covid_df = vaccinations_data[merge_mask].merge(
    covid_data,
    on=['date','country'],
    how='right'
)
display(covid_df.head())


,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
0,Afghanistan,2020-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN
1,Afghanistan,2020-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Afghanistan,2020-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Afghanistan,2020-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Afghanistan,2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86785 entries, 0 to 86784
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              86785 non-null  object        
 1   date                                 86785 non-null  datetime64[ns]
 2   total_vaccinations                   10629 non-null  float64       
 3   people_vaccinated                    10007 non-null  float64       
 4   people_vaccinated_per_hundred        10007 non-null  float64       
 5   people_fully_vaccinated              7695 non-null   float64       
 6   people_fully_vaccinated_per_hundred  7695 non-null   float64       
 7   daily_vaccinations                   18264 non-null  float64       
 8   vaccines                             18451 non-null  object        
 9   confirmed                            86785 non-null  float64       
 10  deaths    

In [11]:
covid_df['death_rate'] = covid_df['deaths'] / covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered'] / covid_df['confirmed'] * 100

In [12]:
display(covid_df.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,death_rate,recover_rate
0,Afghanistan,2020-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0
1,Afghanistan,2020-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,2020-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,2020-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
display(
    covid_df['death_rate']
    [covid_df['country'] == 'United States'].max().round(2)
    )

display(
    covid_df['recover_rate']
    [covid_df['country'] == 'Russia'].mean().round(2)
    )

7.19

67.06

In [14]:
grouped_cases = covid_df.groupby('date')['daily_confirmed'].sum()
grouped_cases.plot(
    kind='line',
    figsize=(12, 4),
    title='Ежедневная заболеваемость во времени',
    grid = True,
    lw=3
);

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.